# **Inference**
Kode berikut digunakan untuk menggunakan model yang sudah di latih untuk memprediksi hasil dari data baru yang belum pernah dilihat sebelumnya

## **Import Library**

In [19]:

import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

## **Load Keras model, svm model, tokenizer, dan label, TF-IDF vectorizer**

In [20]:
model_biLSTM = load_model('../models/model_biLSTM.h5')
model_biGRU = load_model('../models/model_biGRU.h5')

with open('../models/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

with open('../models/label_mapping.pkl', 'rb') as f:
    label_mapping = pickle.load(f)

# Buat reverse mapping (0 → 'negative', dst.)
reverse_label_mapping = {v: k for k, v in label_mapping.items()}

# Load SVM and TF-IDF vectorizer
with open('../models/svm_model.pkl', 'rb') as f:
    svm_model = pickle.load(f)

with open('../models/tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

## **Fungsi Prediksi**

In [21]:
# Fungsi prediksi BiLSTM/GRU
def predict_sentiment_keras(model, sentence, max_len=100):
    sequence = tokenizer.texts_to_sequences([sentence])
    padded = pad_sequences(sequence, maxlen=max_len, padding='post')
    pred = model.predict(padded)
    predicted_class = pred.argmax(axis=-1)[0]
    return reverse_label_mapping[predicted_class]

# Fungsi prediksi SVM
def predict_sentiment_svm(sentence):
    transformed = tfidf_vectorizer.transform([sentence])
    return svm_model.predict(transformed)[0]

## **Uji prediksi**

In [22]:
# Kalimat baru yang ingin diuji
kalimat_baru = [
    "fitur banyak errornya, kamera jelek buram",
    "Senang sekali pembayaran lebih mudah",
    "Biasa saja"
]

# Uji prediksi untuk masing-masing kalimat baru
print("-" * 30)
for kalimat in kalimat_baru:
    print(f"Kalimat: {kalimat}")
    print("biLSTM:", predict_sentiment_keras(model_biLSTM, kalimat), "\n")

print("-" * 30)

for kalimat in kalimat_baru:
    print(f"Kalimat: {kalimat}")
    print("biGRU :", predict_sentiment_keras(model_biGRU, kalimat), "\n")

print("-" * 30)

for kalimat in kalimat_baru:
    print(f"Kalimat: {kalimat}")
    print("SVM   :", predict_sentiment_svm(kalimat), "\n")

------------------------------
Kalimat: fitur banyak errornya, kamera jelek buram
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 211ms/step
biLSTM: negative 

Kalimat: Senang sekali pembayaran lebih mudah
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
biLSTM: positive 

Kalimat: Biasa saja
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
biLSTM: neutral 

------------------------------
Kalimat: fitur banyak errornya, kamera jelek buram
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
biGRU : negative 

Kalimat: Senang sekali pembayaran lebih mudah
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
biGRU : positive 

Kalimat: Biasa saja
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
biGRU : neutral 

------------------------------
Kalimat: fitur banyak errornya, kamera jelek buram
SVM   : negative 

Kalimat: Senang sekali pembayaran lebih mudah
SVM   : positive 

Kalimat: Biasa saja
SVM   : neutral 

